## Import datafile from Postgres

Start with a *trial database* named modifiedData1.

When importing from Postgres change path and filename.

In [94]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder, LabelEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint
import keras_tuner as kt
from keras.layers import LeakyReLU

import re

#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv("Resources/modifiedData1.csv")
df.head()

,Date,MMWR_week,Location,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Unk_Manuf,Dist_Per_100K,Distributed_Per_100k_12Plus,Distributed_Per_100k_18Plus,Distributed_Per_100k_65Plus,Administered,Percent,Greater75Percent
0,10/28/21,43,MO,"8,411,045","347,900","3,293,640","4,769,505",0,"137,045","160,647","176,449","791,973","6,724,837",79.952455,1
1,10/28/21,43,UT,"4,355,130","198,100","1,611,740","2,545,290",0,"135,845","168,005","191,453","1,190,340","3,664,969",84.152919,1
2,10/28/21,43,MT,"1,464,145","87,700","619,680","756,765",0,"136,992","159,563","174,264","709,245","1,178,365",80.481441,1
3,10/28/21,43,VA2,"6,744,640","559,700","3,192,860","2,992,080",0,0,0,0,0,"5,974,228",88.577419,1
4,10/28/21,43,SD,"1,324,315","72,100","553,500","698,715",0,"149,698","179,422","198,382","872,000","1,016,852",76.783243,1


Clean data

1.  Drop null vaules in columns
2.  Drop null values in rows

In [95]:
df = df.dropna(axis = "columns", how = "all")

In [96]:
df = df.dropna()

Delete unnecssary columns

In [97]:
df.drop(columns =["MMWR_week","Administered","Percent"], inplace = True)
df.head()

,Date,Location,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Unk_Manuf,Dist_Per_100K,Distributed_Per_100k_12Plus,Distributed_Per_100k_18Plus,Distributed_Per_100k_65Plus,Greater75Percent
0,10/28/21,MO,"8,411,045","347,900","3,293,640","4,769,505",0,"137,045","160,647","176,449","791,973",1
1,10/28/21,UT,"4,355,130","198,100","1,611,740","2,545,290",0,"135,845","168,005","191,453","1,190,340",1
2,10/28/21,MT,"1,464,145","87,700","619,680","756,765",0,"136,992","159,563","174,264","709,245",1
3,10/28/21,VA2,"6,744,640","559,700","3,192,860","2,992,080",0,0,0,0,0,1
4,10/28/21,SD,"1,324,315","72,100","553,500","698,715",0,"149,698","179,422","198,382","872,000",1


Convert Date to string

In [98]:
df["Date"] = df["Date"].astype(str)

Apply Label Encoder to the Dates which are now strings

In [99]:
le = LabelEncoder()
df["Date"] = le.fit_transform(df["Date"])

In [100]:
df.head()

,Date,Location,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Unk_Manuf,Dist_Per_100K,Distributed_Per_100k_12Plus,Distributed_Per_100k_18Plus,Distributed_Per_100k_65Plus,Greater75Percent
0,8,MO,"8,411,045","347,900","3,293,640","4,769,505",0,"137,045","160,647","176,449","791,973",1
1,8,UT,"4,355,130","198,100","1,611,740","2,545,290",0,"135,845","168,005","191,453","1,190,340",1
2,8,MT,"1,464,145","87,700","619,680","756,765",0,"136,992","159,563","174,264","709,245",1
3,8,VA2,"6,744,640","559,700","3,192,860","2,992,080",0,0,0,0,0,1
4,8,SD,"1,324,315","72,100","553,500","698,715",0,"149,698","179,422","198,382","872,000",1


In [101]:
df["Location"].dtypes

dtype('O')

In [89]:

cat = df.dtypes[df.dtypes == "object"].index.to_list
cat

<bound method IndexOpsMixin.tolist of Index(['Location', 'Distributed', 'Distributed_Janssen', 'Distributed_Moderna',
       'Distributed_Pfizer', 'Distributed_Unk_Manuf', 'Dist_Per_100K',
       'Distributed_Per_100k_12Plus', 'Distributed_Per_100k_18Plus',
       'Distributed_Per_100k_65Plus'],
      dtype='object')>

In [102]:
ohe_cols = ['Distributed', 'Distributed_Janssen', 'Distributed_Moderna',
       'Distributed_Pfizer', 'Distributed_Unk_Manuf', 'Dist_Per_100K',
       'Distributed_Per_100k_12Plus', 'Distributed_Per_100k_18Plus',
       'Distributed_Per_100k_65Plus']

for col in ohe_cols:
    df[col] = df[col].astype(str)

In [103]:
ohe_cols = ['Distributed', 'Distributed_Janssen', 'Distributed_Moderna',
       'Distributed_Pfizer', 'Distributed_Unk_Manuf', 'Dist_Per_100K',
       'Distributed_Per_100k_12Plus', 'Distributed_Per_100k_18Plus',
       'Distributed_Per_100k_65Plus']

for col in ohe_cols:
    df[col] = df[col].str.replace(",", "", regex = True)

df.head()

,Date,Location,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Unk_Manuf,Dist_Per_100K,Distributed_Per_100k_12Plus,Distributed_Per_100k_18Plus,Distributed_Per_100k_65Plus,Greater75Percent
0,8,MO,8411045,347900,3293640,4769505,0,137045,160647,176449,791973,1
1,8,UT,4355130,198100,1611740,2545290,0,135845,168005,191453,1190340,1
2,8,MT,1464145,87700,619680,756765,0,136992,159563,174264,709245,1
3,8,VA2,6744640,559700,3192860,2992080,0,0,0,0,0,1
4,8,SD,1324315,72100,553500,698715,0,149698,179422,198382,872000,1


In [104]:
df.dtypes

Date                            int64
Location                       object
Distributed                    object
Distributed_Janssen            object
Distributed_Moderna            object
Distributed_Pfizer             object
Distributed_Unk_Manuf          object
Dist_Per_100K                  object
Distributed_Per_100k_12Plus    object
Distributed_Per_100k_18Plus    object
Distributed_Per_100k_65Plus    object
Greater75Percent                int64
dtype: object

In [108]:
ohe_cols = ['Distributed', 'Distributed_Janssen', 'Distributed_Moderna',
       'Distributed_Pfizer', 'Distributed_Unk_Manuf', 'Dist_Per_100K',
       'Distributed_Per_100k_12Plus','Distributed_Per_100k_18Plus',
       'Distributed_Per_100k_65Plus' ]

for col in ohe_cols:
    df[col] = df[col].astype(float)


In [109]:
df.dtypes

Date                             int64
Location                        object
Distributed                    float64
Distributed_Janssen            float64
Distributed_Moderna            float64
Distributed_Pfizer             float64
Distributed_Unk_Manuf          float64
Dist_Per_100K                  float64
Distributed_Per_100k_12Plus    float64
Distributed_Per_100k_18Plus    float64
Distributed_Per_100k_65Plus    float64
Greater75Percent                 int64
dtype: object

In [110]:
df.head()

,Date,Location,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Unk_Manuf,Dist_Per_100K,Distributed_Per_100k_12Plus,Distributed_Per_100k_18Plus,Distributed_Per_100k_65Plus,Greater75Percent
0,8,MO,8411045.0,347900.0,3293640.0,4769505.0,0.0,137045.0,160647.0,176449.0,791973.0,1
1,8,UT,4355130.0,198100.0,1611740.0,2545290.0,0.0,135845.0,168005.0,191453.0,1190340.0,1
2,8,MT,1464145.0,87700.0,619680.0,756765.0,0.0,136992.0,159563.0,174264.0,709245.0,1
3,8,VA2,6744640.0,559700.0,3192860.0,2992080.0,0.0,0.0,0.0,0.0,0.0,1
4,8,SD,1324315.0,72100.0,553500.0,698715.0,0.0,149698.0,179422.0,198382.0,872000.0,1


Use OneHotEncoder on the Location column

In [114]:
obj_list = df.dtypes[df.dtypes == "object"].index.to_list()
obj_list

['Location']

In [116]:
enc = OneHotEncoder(sparse = False)
encode_df = pd.DataFrame(enc.fit_transform(df[obj_list]))
encode_df.columns = enc.get_feature_names(obj_list)
encode_df

,Location_AK,Location_AL,Location_AR,Location_AS,Location_AZ,Location_BP2,Location_CA,Location_CO,Location_CT,Location_DC,...,Location_US,Location_UT,Location_VA,Location_VA2,Location_VI,Location_VT,Location_WA,Location_WI,Location_WV,Location_WY
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
574,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
576,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
df = df.merge(encode_df, left_index = True, right_index = True)
df = df.drop(obj_list, 1)
df.head()

,Date,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Unk_Manuf,Dist_Per_100K,Distributed_Per_100k_12Plus,Distributed_Per_100k_18Plus,Distributed_Per_100k_65Plus,...,Location_US,Location_UT,Location_VA,Location_VA2,Location_VI,Location_VT,Location_WA,Location_WI,Location_WV,Location_WY
0,8,8411045.0,347900.0,3293640.0,4769505.0,0.0,137045.0,160647.0,176449.0,791973.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8,4355130.0,198100.0,1611740.0,2545290.0,0.0,135845.0,168005.0,191453.0,1190340.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8,1464145.0,87700.0,619680.0,756765.0,0.0,136992.0,159563.0,174264.0,709245.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8,6744640.0,559700.0,3192860.0,2992080.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,1324315.0,72100.0,553500.0,698715.0,0.0,149698.0,179422.0,198382.0,872000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 578 entries, 0 to 577
Data columns (total 76 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Date                         578 non-null    int64  
 1   Distributed                  578 non-null    float64
 2   Distributed_Janssen          578 non-null    float64
 3   Distributed_Moderna          578 non-null    float64
 4   Distributed_Pfizer           578 non-null    float64
 5   Distributed_Unk_Manuf        578 non-null    float64
 6   Dist_Per_100K                578 non-null    float64
 7   Distributed_Per_100k_12Plus  578 non-null    float64
 8   Distributed_Per_100k_18Plus  578 non-null    float64
 9   Distributed_Per_100k_65Plus  578 non-null    float64
 10  Greater75Percent             578 non-null    int64  
 11  Location_AK                  578 non-null    float64
 12  Location_AL                  578 non-null    float64
 13  Location_AR         

Target column is "Greater75Percent"

This means that the total Administered out of Distributed for a Location on a Date is 75% or greater
(1 is yes, 0 is no).

Other datacleaning considerations could be done;however, the focus at this point will be to set up machnine learning models.

First:  a neural network.

In [119]:
# Split our preprocessed data into our features and target arrays
y = df['Greater75Percent'].values
X = df.drop(columns = ['Greater75Percent']).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [120]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [121]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_feat = len(X[0])
hidden_nodes_layer1=200
hidden_nodes_layer2=50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = number_input_feat, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               15200     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                10050     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 25,301
Trainable params: 25,301
Non-trainable params: 0
_________________________________________________________________


In [122]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [123]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [124]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq= 20000)

In [125]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 15, callbacks=[cp_callback])

Epoch 1/15
14/14 [==============================] - 1s 2ms/step - loss: 0.4259 - accuracy: 0.8083
Epoch 2/15
14/14 [==============================] - 0s 3ms/step - loss: 0.2297 - accuracy: 0.9053
Epoch 3/15
14/14 [==============================] - 0s 3ms/step - loss: 0.1445 - accuracy: 0.9053
Epoch 4/15
14/14 [==============================] - 0s 3ms/step - loss: 0.0974 - accuracy: 0.9723
Epoch 5/15
14/14 [==============================] - 0s 3ms/step - loss: 0.0740 - accuracy: 0.9792
Epoch 6/15
14/14 [==============================] - 0s 3ms/step - loss: 0.0634 - accuracy: 0.9792
Epoch 7/15
14/14 [==============================] - 0s 3ms/step - loss: 0.0563 - accuracy: 0.9769
Epoch 8/15
14/14 [==============================] - 0s 3ms/step - loss: 0.0537 - accuracy: 0.9769
Epoch 9/15
14/14 [==============================] - 0s 2ms/step - loss: 0.0522 - accuracy: 0.9769
Epoch 10/15
14/14 [==============================] - 0s 2ms/step - loss: 0.0512 - accuracy: 0.9769
Epoch 11/15
14/14 [

In [126]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

5/5 - 0s - loss: 0.0520 - accuracy: 0.9724
Loss: 0.052037257701158524, Accuracy: 0.9724137783050537


In [127]:
nn.save("AlphabetSoupCharity.h5")